In [175]:
import pprint
import numpy as np
import pandas as pd
import knn_images
from IPython.display import display as disp
import matplotlib.pyplot as plt


In [288]:
def get_train_data(numBatches):
    train_data = []
    train_labels = []
    file = "cifar-10-batches-py/data_batch_" + str(numBatches)
    batch_data_2 = knn_images.unpickle(file)
    train_data = batch_data_2[b'data']
    train_labels = batch_data_2[b'labels']

    for i in range(1,numBatches):
        file = "cifar-10-batches-py/data_batch_" + str(i)
        batch_data = knn_images.unpickle(file)
        data = batch_data[b'data']
        labels = batch_data[b'labels']
        
        train_data = np.concatenate((train_data, data),0)
        train_labels= np.concatenate((train_labels, labels),0)
        
    return train_data, train_labels

def get_test_data():
    file = "cifar-10-batches-py/test_batch"
    test_batch = knn_images.unpickle(file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']
    return test_data, test_labels

In [289]:
# retrieve train_data, train_labels
train_data, train_labels = get_train_data(5)
test_data, test_labels = get_test_data()

In [282]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

def fit_knn(train_data, train_labels, k):
    knn = KNeighborsClassifier(1)
    knn.fit(train_data,train_labels) #fit data in classifier
    return knn


In [ ]:
#retrieve knn model with specified number of k
knn = fit_knn(train_data, train_labels, 10)
test_data = test_data[:100]
test_labels = test_labels[:100]
pred_labels = knn.predict(test_data)
acc = 0
for i in range(0,len(pred_labels)):
    if pred_labels[i] == test_labels[i]:
        acc += 1
        
print(acc)
from sklearn.metrics import classification_report, confusion_matrix  
# print(confusion_matrix(test_data, pred_labels))  
# print(classification_report(test_data, pred_labels))  
